# Image Classification ML Assignment 

The goal of this assignment is to learn fundamentals of machine learning and image analysis. You will analyze images from Happy Whale, an organization that tracks whales globally, and modify a classifier for these images. 

Part 1: Walk through (Due Feb 27)
- In class, we will walk through the provided code
- This script has several functions that are written for you. For part 1, please do NOT modify any code unless it specifies to change it.
- Answer the questions as you go through the code

Part 2: Make modifications (Due Feb 27)
- Change model parameters or other model aspects at least 3 times 
- Describe what happened when you made these changes

Part 3: Create a team, get inspired and try a new approach to your model (Due March 6)
- Have a group meeting and record your meeting with audio or video (you can record with zoom!)
    - Start by discussing part 2 of the assignment:
        - What approaches you each took to change the ML model in part 2
        - What are similarities and differences between your approaches 
        - What types of changes improved model performance
        - What types of changes made model performance worse
        - Based on the whole group's work, what would y'all want to do next in terms of model optimization?
    - Next, look at code and approaches from the [Happy Whales competition leaderboard](https://www.kaggle.com/competitions/happy-whale-and-dolphin/leaderboard).
        - Is there any strategy that someone used that you could implement? 
            - Ex: learning rate value, # epochs, transformation strategy, or more complex changes to the models
        - Are there strategies that you are interested in learning about?
    - Lastly, make a plan for 1 more round of model optimization
        - Assign each member something to attempt for training the model. Use your cumulative experience to decide the most important things to test. 
        - Group members can make as small as changing the value for a parameter or as big as changing the model.
- Have each member complete their test
- Share results with each other
- Write a 1-3 paragraph summary on your group's attempts and recommendations moving forward
- For Part 3: Turn in the audio/video recording of the group meeting, each member's code (uploaded to their original git repo), and the team's 1-3 paragraph summary over canvas.

### Your choices on data
You can chose between 3 datasets for this assignment. This jupyter notebook is set up using the Species dataset. If you change the dataset, you will need to change the number of classes parameter for the model.
- Species = you will classify images by species (gray, killer, beluga, humpback, false killer, and common dolphin)
  - File path: /projects/bgmp/shared/Bi625/ML_Assignment/Datasets/Whale_species/species
- Species_all = you will classify images by species (minke, gray, killer, beluga, humpback, false killer, and bottlenose and common dolphin). There are a high number of images for bottlenose dolphins and minke whales which leads to this dataset being quite large. Due to the size of the dataset, it has a longer training time than the "Species" dataset.
  - File path: /projects/bgmp/shared/Bi625/ML_Assignment/Datasets/Whale_species/species_all
  - Num classes: 8
- Individuals = you will classify images by individuals. This is a much more challenging classification task because you have different individuals from different species. However, this was the original goal of the Happy Whale competition, so try challenging yourself to this important task!
  - File path: /projects/bgmp/shared/Bi625/ML_Assignment/Datasets/Whale_species/individuals
  - Num classes: 110

You are NOT graded by the success of your model, so you are free to train the model on whichever dataset is more interesting to you. However, **everyone on your team for part 3 should be using the same dataset**.

### Part 1

In [1]:
## Importing packages - Please DO NOT alter this box ##
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data.sampler import WeightedRandomSampler
torch.manual_seed(0)

from captum.attr import IntegratedGradients
from captum.attr import DeepLift
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz

import torchvision
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

import os
import imageio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
from scipy.ndimage import binary_erosion
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

import wandb #comment this out if you are not using weights and biases
import random #comment this out if you are not using weights and biases

# Set device to cuda if it's available otherwise default to "cpu"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

KeyboardInterrupt: 

### Loading in the dataset

By default, this will load in the species dataset, this is the smallest dataset (i.e., it will be relatively fast to train) and has a simple goal (i.e., you will be able to create an accurate model). If you want a challenge, I highly encourage you to try the Species_all (a dataset with 2 additional species) or the individual dataset (a dataset where whales are characterized by individual rather than species).

In [ ]:
images = "/projects/bgmp/shared/Bi625/ML_Assignment/Datasets/Whale_species/species"
wandb.__version__
wandb.login()

In [ ]:
# We must turn our images into tensors, resize, and normalize them 
# We can also add additional transformations to images 
transform = transforms.Compose([transforms.ToTensor(), 
                                #We will start with a model called Resnet18 that is optimized for 224x224 images
                                #It is set to a very SMALL size initially so the model will train fast in class
                                transforms.Resize([32,32]),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize the data, these are the values that ResNet suggests based on their training data (natural scences)
                               ])
all_images = datasets.ImageFolder(images, transform)
print(all_images)

<font color='magenta'>What is the purpose(s) of transforming image data? (2 pts)

The goal with the transformation is that each of the different images can be compared to eachother. For example, if the images are different sizes or if they were taken with different cameras or filters, this step hopefully helps remove some of those biases.

<font color='green'>The below code completes the image transformations as coded in the above box.

In [ ]:
all_images = datasets.ImageFolder(images, transform )
print(len(all_images))
print(all_images)

Our classification labels will be converted to indices, we can determine which indices is which label using the below code

In [ ]:
print(all_images.class_to_idx)

<font color='green'>The below function is used to randomly select dataset images to evaluate how the transformations altered the images.

In [ ]:
def show_random_dataset_image(dataset):
    idx = np.random.randint(0, len(dataset))    # take a random sample
    img, mask = dataset[idx]                    # get the image and the nuclei masks
    f, axarr = plt.subplots(1, 2)               # make two plots on one figure
    axarr[0].imshow(img[0], cmap="viridis")                     # show the image, cmap is the color map that the image is being shown in
    #axarr[1].imshow(mask[0])                    # show the masks
    _ = [ax.axis('off') for ax in axarr]        # remove the axes
    print('Image size is %s' % {img[0].shape})
    print(img.shape)
    plt.show()

In [ ]:
show_random_dataset_image(all_images)

<font color='magenta'>What is the difference between image size and torch image size? (2 pts)

The torch image size not only includes the width and height, but also includes the value 3, which stands for number of channels and in this case it is 3 for rgb.

<font color='magenta'>Add a new transformation to the training data. After your first run of this model during class time, resize the image to 224x224 pixels. (2 pts)

In [ ]:
## ADD YOUR TRANSFORMATION HERE
transform = transforms.Compose([
            transforms.Resize([224,224]), # Resize the image as our model is optimized for 224x224 pixels
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # Normalize the data, these are the values that ResNet suggests based on their training data (natural scences))

all_images = datasets.ImageFolder(images, transform )
show_random_dataset_image(all_images)

<font color='magenta'>How did your transformation impact the images? (1 pt)

This transformation made this much more clearer!

### Creating datasets used in training and testing

After loading in the data, we separate the data into training, validation, and testing datasets. The training data is inputted into the model during the training phase, and the model's predictions from this data is used to modulate the weights of the model. The validation data is used while the model trains, but the model is in a evaluation rather than training mode. The validation data gives us a real time view on how accurately the model is predicting our labels. The testing data is used after the model has completed training and tells us how successful our model will be with a novel dataset.

Once creating these 3 datasets, we prepare dataloaders which are used to load the images into our models. These images are loaded in batches into the model (a parameter called batch size). Because our dataset is imbalanced, we use a weighted random sampler to select images for our batches in the dataloader. Therefore to create our dataloader, we start by creating a function to get weights, calculating weights, initiating our weighted random sampler, and then creating our dataloaders.

<font color='green'>The below code eastablishes the data split between training, validation, and testing.

In [ ]:
train_size = int(0.7 * len(all_images))
val_size = int(0.15 * len(all_images))
test_size = len(all_images) - (train_size + val_size)
print(train_size, val_size, test_size)
assert train_size + val_size + test_size == len(all_images)

In [ ]:
train_set, val_set, test_set = torch.utils.data.random_split(all_images, [train_size, val_size, test_size])

<font color='green'>The below function is used to get weights for the image classes.

In [ ]:
def _get_weights(subset,full_dataset):
    ys = np.array([y for _, y in subset])
    counts = np.bincount(ys)
    label_weights = 1.0 / counts
    weights = label_weights[ys]

    print("Number of images per class:")
    for c, n, w in zip(full_dataset.classes, counts, label_weights):
        print(f"\t{c}:\tn={n}\tweight={w}")
        
    return weights

In [ ]:
train_weights = _get_weights(train_set,all_images)
train_sampler = WeightedRandomSampler(train_weights, len(train_weights))

<font color='magenta'>What is the impact of the class weight on how data is loaded into the model? What does it mean if a class has a low vs high weight? (2 pts)

<details>
  <summary>Hint</summary>
  
Check out this resource: https://towardsdatascience.com/demystifying-pytorchs-weightedrandomsampler-by-example-a68aceccb452 


The weights indicate the probability of an image being selected. Classes that have more pictures that belong to that class will have smaller weights. That is why humpback whales have the lowest weight. 

In [ ]:
train_loader = DataLoader(train_set, batch_size=48, drop_last=True, sampler=train_sampler)
val_loader = DataLoader(val_set, batch_size=48, drop_last=True, shuffle=True)
test_loader = DataLoader(test_set, batch_size=48, drop_last=True, shuffle=True)

### Picking hyperparameters

We can set a few hyperparameters that we decide and can tune. These include:

- Learning rate = how much to update our model's parameters at each batch/epoch
- batch size = the number of data samples to pass through the network before updating it
    - If you change the batchsize below, make sure you ALSO change it in the commands above for the DataLoader
- Number of epochs = # of times to iterate over the dataset in model training

In [ ]:
learning_rate=1e-3
batchsize=48
epochs=5

<font color='magenta'>How would model training be impacted if we decreased our learning rate? What about if we increased it? (2 pts)

Increasing the learning rate might take the model longer as the model's parameters will update much more frequently during the learning process. If we decrease the learning rate, it will run faster, but maybe not have the same amount of predicting power than if it were more frequently updating those parameters.

### Training our model

For part 1, we are going to use the model ResNet18. This is a published model for imaging data. Resnet18 is built from 4 residual blocks with two convolutional layers (Conv2d). Each convolutional layer is followed by a batch normalization step (BatchNorm2D) and a ReLU activation layer (ReLU). There is also a shortcut connection which adds input to output and allows the network to learn residual mapping.

I provided the complete code for the ResNet18 model. In the future, if you want to use resnet18, you can load it simply by running `resnet18_model = torchvision.models.resnet18(weights = False, progress  = True, num_classes=XXX)`. We will look at the full code for the model so you can see how this model was created.  

<font color='green'>The below code builds the residual blocks that ResNet18 uses in the model.

In [ ]:
## First, we create the basic block that will be used in our residual net
class BasicBlock(nn.Module):
    # Initializing method for the basic block (It's OOP!)
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        ## Conv1: convolution layer, batch normalization, ReLU activation
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        ## Conv2: convolution layer, batch normalization
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        ## Shortcut connection: adds input to output
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    # The forward method calls on each layer
    def forward(self, x):
        ## Conv1: convolution layer, batch normalization, ReLU activation
        out = self.conv1(x) 
        out = self.bn1(out)
        out = self.relu(out)
        ## Conv2: convolution layer, batch normalization
        out = self.conv2(out)
        out = self.bn2(out)
        ## Shortcut connection
        out += self.shortcut(x)
        ## Final activation
        out = self.relu(out)
        return out

<font color='green'>The below code builds and initializes the ResNet18 model.

In [ ]:
## Next, we put together these building blocks and create our residual net
class ResNet18(nn.Module):
    # specify the number of classes that we are predicting
    def __init__(self, num_classes=6):
        super(ResNet18, self).__init__()
        # In channels = Num pixels in H + Num pixels in W
        self.in_channels = 64
        
        # First convolution set (convolution, batch norm, relu)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # Our building blocks
        # The numbers correspond to the matrix shape
        ### We increase the number of filters/channels (i.e., the first number) as we go
        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)

        # Average pooling 
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Final layer that makes the classification
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        # First convolution set (convolution, batch norm, relu)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        # Our 4 building blocks
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        # Final layer that makes the classification
        out = self.fc(out)
        return out

## Getting our model and transferring it to the GPU
model = ResNet18().to(device)
print(model)

<font color='green'>The below code eastablishes the loss function and optimizer used in training the model.

In [ ]:
#define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

<font color='green'>The below functions are used to train, evaluate, and test the Machine Learning model.

In [ ]:
wandb.init(
    project="BGMP_HappyWhale",
    name="Makayla-HappyWhale-SpeciesDataset-hyperparameters", ### Update with your name!
    config={"learning rate":0.001,
        "architecture": "CNN",
        "dataset": "Species",
        "epochs":5}
)

num_epochs = epochs
train_losses, train_acc_list, val_losses, val_acc_list = [], [], [],[]

for epoch in range(num_epochs):
    # Setting model to "training mode"
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    batch=0

    # For each batch of data within the loader
    for inputs, labels in train_loader:
        # Send our input images and their labels to the GPU
        inputs, labels = inputs.to(device), labels.to(device)
        # Zero the gradients
        optimizer.zero_grad()
        # Inputting our training images into the model
        # and Predicting the image classification label
        outputs = model(inputs)
        # Figuring out the loss from our predictions
        loss = criterion(outputs, labels)
        # Compute gradients (aka backward pass)
        loss.backward()
        # Update model parameters
        optimizer.step()

        # Adding the loss to our running sum
        # Loss is calculated for each batch within an epoch
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        batch+=1
        print(f'Epoch [{epoch+1}/{num_epochs}] | Batch #{batch} | Batch Accuracy {(correct/total)*100:.2f}%')


    # Getting metrics for our training pass 
    train_loss = running_loss / len(train_loader.dataset)
    train_acc = 100. * correct / total
    train_losses.append(train_loss)
    train_acc_list.append(train_acc)
    
    # Switching our model to "evaluation mode"
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0
    # Disable gradient calculation b/c we are evalulating the model
    with torch.no_grad():
        # Load in batches of our validation data
        for inputs, labels in val_loader:
            # Send test images and labels to the GPU
            inputs, labels = inputs.to(device), labels.to(device)
            # Predict the image classification label
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            # Figuring out how many predicted labels = true labels
            correct += predicted.eq(labels).sum().item()

            # Figuring out the loss from our predictions
            loss = criterion(outputs, labels)
            # Adding the loss to our running sum
            # Loss is calculated for each batch within an epoch
            running_loss += loss.item() * inputs.size(0)
    # Getting our accuracy from our test data
    val_acc = 100. * correct / total
    val_acc_list.append(val_acc)
    # Getting the loss from our test data
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    
    print(f'Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}% | Val Loss: {val_loss:.2f}%')

    # log metrics to wandb
    wandb.log({"validation_accuracy": val_acc, "validation_loss": val_loss, "train_loss":train_loss})

wandb.finish()

#### For our class training, run the below model and consider the success of the model training. 

<font color='magenta'>What is your starting accuracy and final accuracy? What happened to the model to change the accuracy from epoch 1 to 5? (2 pts)

The starting accuracy was ~19% to start. As the model started, the batch accuracy percent initially dips before increasing after the 2nd batch. On epoch 2 batch 20 we are up to over 60%. The run ended with a validation accuracy of 66% with a validation loss of 0.83 and a training loss of 0.8. The final batch accuracy was 70.78%. Overall, as the model learned through each epoch, the model accuracy improved. The most improvement occured during the first few batches of the first epoch. 

In [ ]:
plt.plot(range(num_epochs), val_acc_list, color = "magenta")
plt.xlabel("Epoch number")
plt.ylabel("Valiation accuracy")
plt.show()  

plt.plot(range(num_epochs), val_losses, color = "purple")
plt.xlabel("Epoch number")
plt.ylabel("Valiation loss")
plt.show() 

<font color='green'>After running the model and calculating the overall accuracy, we can examine the data via a confusion matrix which highlights the accuracy by class.

In [ ]:
# predict the test dataset
def predict(model, dataset):
    dataset_prediction = []
    dataset_groundtruth = []
    model = model
    with torch.no_grad():
        for x, y_true in dataset:
            inp = x[None]
            y_pred = model(inp)
            dataset_prediction.append(y_pred.argmax().cpu().numpy())
            dataset_groundtruth.append(y_true)
    
    return np.array(dataset_prediction), np.array(dataset_groundtruth)
            
    # create seaborn heatmap with required labels
    ax=sns.heatmap(cm, annot=annot, fmt='', vmax=30, xticklabels=x_axis_labels, yticklabels=y_axis_labels)
    ax.set_title(title)

# Plot confusion matrix 
# orginally from Runqi Yang; 
# see https://gist.github.com/hitvoice/36cf44689065ca9b927431546381a3f7
def cm_analysis(y_true, y_pred, title, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    labels = ['Beluga','Common dolphin', 'False killer whale', 'Fin whale', 'Gray whale','Humpback whale']
    cm = confusion_matrix(y_true, y_pred)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
                              
    x_axis_labels = ['Beluga','Common dolphin', 'False killer whale', 'Fin whale', 'Gray whale','Humpback whale'] # labels for x-axis
    y_axis_labels = ['Beluga','Common dolphin', 'False killer whale', 'Fin whale', 'Gray whale','Humpback whale'] # labels for y-axis
    ax=sns.heatmap(cm, annot=annot, fmt='', vmax=30, xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap = "viridis")
    ax.set_title(title)

## This plot only contains the test set of data
## The test set of data has not been seen by the model yet
y_pred, y_true = predict(model, test_set)

print(y_true)
cm_analysis(y_true, y_pred, "Confusion matrix")

### What features did the model use to make decisions?

Based on the confusion matrices and validation loss, it is clear that the model is learning. To determine the features that are involved in decision making, there are a few possible approaches. One options is shown below, integrated gradients. 

#### Understanding feature importance using Integrated Gradients

<font color='green'>Integrated gradients is an attribution method. Attribution methods score the input data based on the prediction that the model makes using scores for each feature. The gradient is the signal that tells the network how much to increase or decrease a certain weight in the network during backpropogation. These gradients are overlaid onto the images to showcase the regions on the images that influence the weights.

Run the below code. 

In [ ]:
def visualize_ig(idx, 
                 _train_dataset=train_set, 
                 _test_dataset=test_set):
        
    # Get corresponding input and target tensors:
    input_tensor = _test_dataset[idx][0].unsqueeze(0)
    input_tensor.requires_grad = True
    target = _test_dataset[idx][1]
    
    # We will use the IntegratedGradients algorithm:
    algorithm = IntegratedGradients(model)
    
    # First we clear the gradients from the model:
    model.zero_grad()
    
    # Run attribution:
    attr_ig, delta = algorithm.attribute(input_tensor,
                                          target=target,
                                          baselines=input_tensor * 0,
                                          return_convergence_delta=True
                                        )
    
    # Integrated Gradients:
    attr_ig = np.transpose(attr_ig[0].cpu().detach().numpy(), (1, 2, 0))
    
    # Original image:
    original_image = np.transpose((_test_dataset[idx][0].detach().numpy() * 0.5) + 0.5, (1, 2, 0))
    
    print("Annotated whale species", test_true[idx])
    plt.imshow(original_image)
    
    
    print("Predicted whale species", test_pred[idx])


    # This visualises the attribution of labels to 
    viz.visualize_image_attr(attr_ig, 
                             original_image, 
                             method="blended_heat_map",
                             sign="all",
                             show_colorbar=True, 
                             title="Overlayed Integrated Gradients")

test_pred, test_true = predict(model, test_set)

visualize_ig(0)
visualize_ig(8)
visualize_ig(20)
visualize_ig(32)

<font color='magenta'>Does the integrated gradient show what types of features the model used to classify the images? Do you notice any potential problems? (2 pts)

I think the gradietns seem to be identifying the fin outlines from the dark shapes in each picture. Then it uses the fin shape to identify. However, in the first picture, it seems like the model may have identified a group of trees in the background as the whale instead of the part of the image which actually shows the whale. To be fair, the part of the image with the whale is very small as the whale is barely poiking out of the water and that image might be extremely difficult to ID the species. Regardless, the trees should not be representative of the whale species in that first picture.

## Part 2 

Now that you have a sense for how to train a machine learning model, your task is to improve the success of our classification model.

**In three separate training events, make one (or more) modifications to the model training process, re-train the model, and report out your changes and whether they increased model accuracy. Why did you chose your particular change (curiousity, hypothesis, ect.)? Do you have ideas why your modifications were/were not successful?**

Types of changes that you can make include:
- [Using the pre-trained weights for ResNet18 (transfer learning)](https://docs.pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) 
- Changing hyperparameters (learning rate, batch size)
- [Changing transformation approach](https://docs.pytorch.org/vision/0.11/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py)
- [Changing the optimizer](https://docs.pytorch.org/docs/stable/optim.html)
- [Changing the model to a different pre-made model architecture](https://docs.pytorch.org/vision/main/models.html)
- Modifying the ResNet18 model
- Coming up with a whole new model!

If you decide to change any lines of code that are not in the below box, please add them to the box (DO NOT MODIFY ANY CODE ABOVE THIS MARKDOWN BOX - IT MAY BREAK PART 1). 

If your changes are ambitious, the code may take hours to run. If this is the case, please consider turning this notebook into a python script and running it via sBATCH. 

**Please note, you will NOT be graded by the success of your classifiers! It is okay if you are unable to improve the accuracy of the classifier**

Just as a quick reminder, here is the confusion matrix and graph results from the first model. I can use these to compare my model 1 to these.
![`model1_1 accuracy`](./model1_1/model1_1_accuracy.png)
![`model1_1 val loss`](./model1_1/model1_1_validation_loss.png)
![`model1_1 confusion matrix`](./model1_1/ConfusionMatrix1_1.png)

#### Your Model 1 (10 pts)

<font color='magenta'>Describe your changes to the model training process (either changing something about the model, a hyperparameter, how data was inputted into the model, or another aspect) and why you selected those changes.

All 3 of my models were run in python scripts and can be found on my repository. For the first 3 models, I kept the resolutions of the images 32x32. (All files in respect to each of these models can be found in the directories labeled model1, model2, model3, and model4 on the repository.) This is because I expect these to run a little faster than a more higher resolution photo like 244x244.
In the first model, I set batch size to 48, learning rate to 0.001, and epoch to 5. This should mimic what I did in part 1, but with a lower resolution, so I expect it to be a little less computationally heavy.

<font color='magenta'>Were your changes successful? Do you have ideas why your changes were successful or not? 

This was successful in the sense that it was faster (9min instead of over an hour), which was what I was aiming for when reducing the resolution. This also resulted in a lower validation accuracy of 64% rather than 73%.

Model 1 Accuracy 

![model 1 accuracy](./model1/model1_accuracy.png)

Model 1 Validation Loss

![model 1 validation loss](./model1/model1_validation_loss.png)

Model 1 Conusion Matrix

![model 1 confmatrix](./model1/ConfusionMatrix1.png)

This model seems to also have higher true predictions for all whale species compared to the first model with the exception of the false killer whale. It is interesting that not all species were more strongly predicted with one model over the other when this model seemed to be doing so strongly. These predictions could be slightly different the next time I run each of the models as well so maybe that opens up some variability that exists for each time the model is run. The general shape of the loss and accuracy plots look similar between the two models, but running more epochs might allow for more fine tuning of the model.

#### Your Model 2 (10pts)

<font color='magenta'>Describe your changes to the model training process (either changing something about the model, a hyperparameter, how data was inputted into the model, or another aspect) and why you selected those changes.

Still with a resolution of 32x32, I set the following parameters: batchsize = 60, learning rate = 1e-3, and epochs = 7. Overall, I increased the batch size and the epochs in hopes that this is a more robust, but likely also more computationally heavy model. I expect the accuracies to be higher than those seen in model 1.

<font color='magenta'>Were your changes successful? Do you have ideas why your changes were successful or not? 


Model 2 Accuracy 

![model 2 accuracy](./model2/model2_accuracy.png)

Model 2 Validation Loss

![model 2 validation loss](./model2/model2_validation_loss.png)

Model 2 Conusion Matrix

![model 2 confmatrix](./model2/ConfusionMatrix2.png)

This model (#2) was extremely successful. Model 2 had a low resolution, but with a higher batch size and higher epoch number, the model had more opportunities to train. This led to a validation accuracy of 85%. Though I will foreshadow that I re-ran model 2 with the higher resolution later (called model 2_1), but this instead resulted in a lower validation accuracy of 66%, which is well below the first model's validation accuracy. In fact, all of them were like that. The lower resolution versions of the models had higher validation accuracies. I will address this in my final conclusion. 

In the confusion matrix, we see that this model does better with some species, but not all species compared to the last mode (model 1). It seems to do a little more poorly with identifying common dolfin, fin whale, and gray whale species. 

There is more epochs for this model so it has more time to learn and adapt as seen in both the accuracy and validation loss graphs compared to the last model too. Overall, this can take time to run so the next model is the first that simply aims at reducing run time instead of improving accuracy. This was initially done to troubleshoot the issue I was having generating the graphs like the model accuracy, validation loss, and confusion matrix figures like those seen above.

#### Your Model 3 (10 pts)

<font color='magenta'>Describe your changes to the model training process (either changing something about the model, a hyperparameter, how data was inputted into the model, or another aspect) and why you selected those changes.

Still with a resolution of 32x32, I set the following parameters: batchsize = 10, learning rate = 1e-5, and epochs = 3. This was my first attempt at running a faster version of the model to see how much validation accuracy I would lose (and eventually use this to become a test run for printing the corresponding graphs like the confusion matrix (this is my precursor to model 4, which is my fastest model)). With the goal of reducing time, the batchsize and epochs were lowered. However, lowering the batch size may take longer to parse through all of the images in the dataset as there will be more batches now per epoch. The learning rate was lowered too, but in retrospect, this may needed to have been increased to more agressively adjust when there were fewer epochs, but again, there were more batches per epoch so this may balance out the tradeoffs and lead to a similar percent validation accuracy.  

<font color='magenta'>Were your changes successful? Do you have ideas why your changes were successful or not? 

Model 3 Accuracy 

![model 3 accuracy](./model3/model3_accuracy.png)

Model 2 Validation Loss

![model 3 validation loss](./model3/model3_validation_loss.png)

Model 3 Conusion Matrix

![model 3 confmatrix](./model3/ConfusionMatrix3.png)

The changes made to model 3 definitely reduced the runtime, but actually not enough to be faster than model 1. As I mentioned earlier, this may be because setting a batch size of 10 increases the amount of batches needed to  be run for each epoch. The validation accuracy is lower than model 2 as this model has a validation accuracy of 77%, which is less than 86%. However, it is a higher validation accuracy compared to model 1. This again might just be due to more time training and the batch size of 10 actually might increase model training time.

We can see that Model 3 has some lower true calls in the confusion matrix than model 2 in all but the fin whale and the gray whale. The shapes in the accuracy and validation loss graphs are pretty similar across all 3 models. For example the accuracy plots have a general increase while the validation loss has a general decrease. This is likely because the number of epochs is too small and we might see more variation with higher epochs. 

<font color='magenta'>Summarize your 3 steps, their success, and what you learned (5 pts)

I performed my models by running python scripts for each.  I have made a table below for each of the models and epochs. I was able to fine tune model 4 to run very quickly for troubleshooting purposes. 

All models were run with 60G memory, 10 cores, and patition = gpu. Time was also set to 10 hours, but none of the models ever came close to that time. 

Here is a comparison table of all 4 of my runs:
| Model Number | resolution | batch size| learning rate | epochs | Train Loss | Validation Accuracy | Validation Loss | Runtime|
| --- | --- | --- | --- |--- | --- | --- | --- |--- |
|Model 1| 32x32 | 48 | 1e-3 | 5 | 0.4408 | 73.96% | 0.6561 | 9m 22s|
|Model 1_1*| 244x244 | 48 | 1e-3 | 5 | 0.7991 | 64.84% | 0.90 | 1h 20m 29s|
|Model 2| 32x32 |60 | 1e-3 | 7 |0.3251 | 85.8333% |  0.3385 |12m 27s|
|Model 2_1| 244x244 |60 | 1e-3 | 7 | 0.6712| 65.83% | 0.8834 |1h 41m 40s|
|Model 3| 32x32 |10 | 1e-5 | 3 | 0.4878| 76.82% | 0.6347 |9m 40s|
|Model 3_1| 244x244 |10 | 1e-5 | 3 | 0.7681| 65.61% | 0.9384 |50m 38s|
|Model 4 |32x32 | 100 | 1e-5 | 2 | 1.051| 29.75%|1.6204|4m 10s| 4m 10s|
|Model 4_1 |244x244 | 100 | 1e-5 | 2 | 0.9634| 41.5% | 1.5921 |28m 14s|

*completed as part 1 in this assignment

It is a little bizarre to me that the higher resolution images have lower validation accuracies. This might mean that more broader types of shapes in each of the images is stronger at predicting the whales. However, this might not be the most reliable with a larger dataset. I would expect that the higher resolution image is more trustworthy to be using for identification, but that it allows the model to identify shapes like background trees and use those to predict the species, which is not reliable and is lowering the validation accuracy. 